In [8]:
!pip install spacy
!pip install nltk
!pip install vaderSentiment
!python -m spacy download en_core_web_sm
!pip install newspaper3k


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-15 10:01:58.533308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-15 10:01:58.533446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_

In [18]:
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from newspaper import Article # Became redundant for this task
import requests
from bs4 import BeautifulSoup

nltk.download('vader_lexicon')

nlp = spacy.load('en_core_web_sm')
sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [23]:
def analyze_article_sentiment(article_link, person_name):
    
    # article = Article(article_link, MAX_SUMMARY = 20 * 1024 * 1024)
    # article.download()
    # article.parse()
    # The above way to extract the text only took a small portion of the article where Trumps name wasn't mentioned
    # Oddly enough it wasn't even the first part of the article, it was about in the middle where it took out a segment.
    # I even tried increasing the downloaded bytes, newspaper3k normally takes 5 * 1024 * 1024


    response = requests.get(article_link)
    soup = BeautifulSoup(response.content, 'html.parser')

    article_text = ''
    for p in soup.find_all('p'):
        article_text += p.text



    
    # Getting the named entities
    doc = nlp(article_text)
    named_entities = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

    # print(doc) Error checking when it didn't find Trumps name

    # Determine the sentiment of the article towards the named entity using VADER
    sentiment_scores = sid.polarity_scores(article_text)

    # Is Trump mentioned check
    if person_name.lower() in " ".join(named_entities).lower():
        # Is he mentioned in a positive or negative way check
        if sentiment_scores["compound"] >= 0:
            return f"The article is positive towards {person_name}."
        else:
            return f"The article is negative towards {person_name}."
    else:
        # print(sentiment_scores) More error  checking
        return f"{person_name} is not mentioned in the article."

    


article_link = "https://www.bbc.com/news/world-us-canada-52012049"
person_name = "Donald Trump"

print(analyze_article_sentiment(article_link, person_name))






The article is negative towards Donald Trump.
